In [12]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter

In [13]:
stopwords=['కూడా', 'ఉన్నారు', 'ఎవరైనా', 'ఎప్పుడు', 'ఎక్కడైనా', 'అందుబాటులో', 'ఒక ప్రక్కన', 'సంబంధం', 'మాత్రమే', 'అడగండి', 'గా', 'మధ్య', 'ప్రకారం', 'అనుమతించు', 'మెచ్చుకో', 'వద్ద', 'తగిన', 'అడ్డంగా', 'అడగడం', 'ఇప్పటికే', 'కనిపిస్తాయి', 'కాదు', 'మరియు', 'మరొక', 'అనుగుణంగా', 'అయితే', 'చేయగలిగింది', 'దాదాపు', 'గురించి', 'పై', 'వెంట', 'వేరుగా', 'చుట్టూ', 'దూరంగా', 'ఏ', 'నిజంగా', 'అనుమతిస్తుంది', 'అందరూ', 'ఎవరో ఒకరు', 'వ్యతిరేకంగా', 'ఏదైనా', 'తర్వాత', 'మళ్ళీ', 'ఏమైనప్పటికి', 'ఒక']

In [14]:
vowels=set("అ ఆ ఇ ఈ ఉ ఊ ఋ ౠ ఌ ౡ ఎ ఏ ఐ ఒ ఓ ఔ అం అః".split())

conso="క	ఖ	గ	ఘ	ఙ	చ	ఛ	జ	ఝ	ఞ	ట	ఠ	డ	ఢ	ణ	త	థ	ద	ధ	న	ప	ఫ	బ	భ	మ	య	ర	ల	వ	శ	ష	స	హ	ళ	క్ష	ఱ"

matra = "ఀ	ఁ	ం	ః	ఄ	ఽ	ా	ి	ీ	ు	ూ	ృ	ౄ	ె	ే	ై	ొ	ో	ౌ	్	ౕ	ౖ	ౢ	ౣ"

halantha='్'

In [15]:
consonants=set(conso.split("\t"))
matra = set(matra.split("\t"))

vowels = list(vowels)
consonants=list(consonants)
matra=list(matra)

alphabets = vowels+consonants

In [16]:
temp=open('pickles/posting_list.pkl',"rb")
posting_lists=pickle.load(temp)

temp=open('pickles/file_idx.pkl','rb')
file_index=pickle.load(temp)

In [17]:
unique_words=set(posting_lists.keys())

In [18]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [20]:
data=pd.read_csv('../telugu.csv')

In [34]:
def brs(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    
    if len(words)>1:
        query_words=[words[0]]
    else:
        query_words=words

    for i in range(1,len(words)):
        if words[i] not in ["మరియు","గాని"]:
            if query_words[-1] not in ["మరియు","గాని"]:
                query_words.append("మరియు")
                query_words.append(words[i])
            else:
                query_words.append(words[i])
        elif query_words[-1] not in ["మరియు","గాని"]:
            query_words.append(words[i])
    
    operators=[]
    main_words=[]
    for w in query_words:
        if w in ["మరియు","గాని"]:
            operators.append(w)
        else:
            main_words.append(w)
    print(query_words)
    print(operators)
    print(main_words)
    
    n=len(file_index)
    word_vector=[]
    word_vector_matrix=[]

    for w in main_words:
        word_vector=[0]*n
        if w in unique_words:
            for x in posting_lists[w].keys():
                word_vector[x]=1
        word_vector_matrix.append(word_vector)
        
        
    for w in operators:
        vector1=word_vector_matrix[0]
        vector2=word_vector_matrix[1]

        if w=="మరియు":
            result=[b1&b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
        else:
            result=[b1|b2 for b1,b2 in zip(vector1,vector2)]

            word_vector_matrix.pop(0)
            word_vector_matrix.pop(0)

            word_vector_matrix.insert(0,result)
    
    
    final_word_vector=word_vector_matrix[0]
    index=0
    slokes=[]
    description=[]
    verse_index=[]
    for i in final_word_vector:
        if not counter:
            break
        if i==1:
            verse_index.append(index)
            counter=counter-1

        index+=1
    return verse_index


In [39]:
data.loc[47]['Commentary']

'ఎప్పుడైనా మనము దుఃఖం లో మునిగిపోయినప్పుడు, మన బుద్ది ఆ దుఃఖానికి మూల కారణాన్ని విశ్లేషిస్తూ ఉంటుంది. ఎపుడైతే ఇక ఎక్కువ ఆలోచించలేదో, అప్పుడు మానసికంగా కుంగిపోవటం మొదలవుతుంది. అర్జునుడి సమస్యలు అతని అల్పమైన బుద్ది కన్నా పెద్దవిగా పరిణమించటంతో, అతన్ని శోక సముద్రం నుండి కాపాడటానికి తనకున్న భౌతిక జ్ఞానం సరిపోదు. శ్రీ కృష్ణుడిని గురువుగా స్వీకరించిన తరువాత తన దయనీయ స్థితిని వెల్లడిచేయుచూ మనస్సు లో ఉన్నదంతా చెప్తున్నాడు, అర్జునుడు.   అర్జునుడి పరిస్థితి అతనొక్కడిదే కాదు. జీవిత ప్రయాణం లో సాగిపోతున్నప్పుడు అప్పుడప్పుడు మనకు ఎదురయ్యేదే. మనకు సంతోషం కావాలి, కానీ దుఃఖం కలుగుతుంటుంది; మనకు జ్ఞానం కావాలి; కాని అజ్ఞానపు మేఘాల్ని తొలగించుకోలేము; పరిశుద్ధమైన ప్రేమని కోరుకుంటాము కాని పదేపదే ఆశాభంగము కలుగుతుంది; మన కాలేజి పట్టాలు, నేర్చుక్కున్న విద్య మరియు లౌకిక పాండిత్యములు జీవితం లో ఎదురయ్యే జటిల సమస్యలకు పరిష్కారం చూపలేవు. మనకు జీవితపు క్లిష్టమైన సమస్యలను పరిష్కరించటానికి ఆధ్యాత్మిక జ్ఞానం అవసరం. మహోన్నత స్థితి లో ఉన్న నిజమైన గురువు లభించినప్పుడు, మనకు వారి నుండి నేర్చుకునే అణకువ, వినయం ఉంటే ఆ యొక్క

In [38]:
verse_index=brs('నాకు చాలా సమస్యలు ఉన్నాయిు')
data.loc[verse_index]

['సమస్యలు']
[]
['సమస్యలు']


,Chapter,Verse,Sloke,Commentary
47,2,8,न हि प्रपश्यामि ममापनुद्या\n\nद्यच्छोकमुच्छोषण...,"ఎప్పుడైనా మనము దుఃఖం లో మునిగిపోయినప్పుడు, మన ..."


In [31]:
text = remove_special_characters('నాకు చాలా సమస్యలు ఉన్నాయి')
words = text.split()
words=[word for word in words if word not in stopwords]
words

['నాకు', 'చాలా', 'సమస్యలు', 'ఉన్నాయి']